# Creating and updating the daily data CSV s

In [2]:
import yfinance as yf
import pandas as pd
import datetime
import os
import schedule
import time

def fetch_daily_data(ticker, start_date, end_date):
    df = yf.download(ticker, start=start_date, end=end_date, interval='1d')
    return df

def update_daily_data_for_ticker(ticker):
    # Determine the CSV file name
    csv_file_name = ticker + '_Daily_Data.csv'

    if not os.path.exists(csv_file_name):
        # Fetch data from '2013-01-01' as initial data
        end_date = datetime.datetime.now().date()
        start_date = '2013-01-01'
        daily_data = fetch_daily_data(ticker, start_date, end_date)
        
        # Reset index and extract date as a separate column
        daily_data.reset_index(inplace=True)
        daily_data['Date'] = daily_data['Date'].dt.date
        
        # Save the DataFrame to a CSV file if the file doesn't exist
        daily_data.to_csv(csv_file_name, index=False)
        print(f"{len(daily_data)} new records added to a new {csv_file_name} file.")
    else:
        # Load existing data from CSV
        existing_df = pd.read_csv(csv_file_name)
        
        # Get the last date from the existing data
        latest_date = pd.to_datetime(existing_df['Date']).max()
        
        # Convert the latest_date to a Pandas Timestamp
        latest_date = pd.Timestamp(latest_date)
        
        # Define date range for fetching new data
        end_date = datetime.datetime.now().date()
        start_date = (latest_date + datetime.timedelta(days=1)).date()
        
        if start_date < end_date:
            # Fetch daily data
            daily_data = fetch_daily_data(ticker, start_date, end_date)
            
            #Reset index and extract date as a separate column
            daily_data.reset_index(inplace=True)
            daily_data['Date'] = daily_data['Date'].dt.date
            
            # Append new data to the CSV file
            daily_data.to_csv(csv_file_name, mode='a', index=False, header=False)
            print(f"{len(daily_data)} new records added to the {csv_file_name} file.")
        else:
            print(f"No new records to add for {ticker}.")

# List of stock symbols
symbols = ["AAPL", "ORCL", "MSFT"]  # Add more symbols as needed

# Function to perform the initial data update for all symbols
def initial_data_update():
    for symbol in symbols:
        update_daily_data_for_ticker(symbol)

# Schedule the automatic update daily
for symbol in symbols:
    schedule.every().day.at("00:10").do(update_daily_data_for_ticker, symbol)

# Perform the initial data update
initial_data_update()

# Run the scheduled tasks
while True:
    schedule.run_pending()
    time.sleep(1)  # Sleep for 24 hours

[*********************100%***********************]  1 of 1 completed

1 Failed download:
- AAPL: No data found for this date range, symbol may be delisted


AttributeError: Can only use .dt accessor with datetimelike values

In [4]:
import pandas as pd

ticker = "AAPL"
csv_file = f"{ticker}_Daily_Data.csv"
df = pd.read_csv(csv_file)

# # Number of rows to delete
# num_rows_to_delete = 1

# # Delete the last N rows
# df = df.iloc[:-num_rows_to_delete]

# # Save the modified DataFrame back to the CSV file
# df.to_csv(csv_file, index=False)

# print("Last", num_rows_to_delete, "rows deleted from the "+ csv_file +" CSV.")

df.tail(10)

,Date,Open,High,Low,Close,Adj Close,Volume
2696,2023-09-19,177.520004,179.630005,177.130005,179.070007,179.070007,51769600
2697,2023-09-20,179.259995,179.699997,175.399994,175.490005,175.490005,58436200
2698,2023-09-21,174.550003,176.300003,173.860001,173.929993,173.929993,63047900
2699,2023-09-22,174.669998,177.078995,174.054993,174.789993,174.789993,56682928
2700,2023-09-22,174.669998,177.078995,174.054993,174.789993,174.789993,56682928
2701,2023-09-25,174.199997,176.970001,174.149994,176.080002,176.080002,46172700
2702,2023-09-26,174.820007,175.199997,171.660004,171.960007,171.960007,64588900
2703,2023-09-27,172.619995,173.039993,169.050003,170.429993,170.429993,66830700
2704,2023-09-28,169.339996,172.029999,167.619995,170.690002,170.690002,56294400
2705,2023-09-29,172.020004,173.070007,170.339996,171.210007,171.210007,51814200


# Adding a price change column to the dataframe 

In [2]:
import pandas as pd

ticker = "MSFT"
df_price = pd.read_csv(f"{ticker}_Daily_Data.csv")

# Calculate the price change compared to the previous date
df_price['Price Change'] = df_price['Close'].diff()

# Add a new column to indicate if the price increased (1) or decreased (0)
df_price['Price Increase'] = (df_price['Price Change'] > 0).astype(int)

df_price.tail()

,Date,Open,High,Low,Close,Adj Close,Volume,Price Change,Price Increase
2680,2023-08-25,321.470001,325.359985,318.799988,322.980011,322.980011,21671400,3.010010,1
2681,2023-08-28,325.660004,326.149994,321.721985,323.820007,323.820007,10588855,0.839996,1
2682,2023-08-29,321.880005,328.983490,321.940002,327.500000,327.500000,10393681,3.679993,1
2683,2023-08-30,328.670013,329.809998,326.450012,328.790009,328.790009,15205500,1.290009,1
2684,2023-08-31,329.200012,330.909912,326.779999,327.950012,327.950012,11526643,-0.839996,0


# Reading News data file

In [3]:
pd.read_csv(f"{ticker}_News_Content.csv").tail()

,title,time_published,summary,overall_sentiment_score,overall_sentiment_label,ticker,relevance_score,ticker_sentiment_score,ticker_sentiment_label
22174,Starfield promises Xbox gamers a lot - and it ...,20230831T160000,Starfield review: This long-awaited Xbox game ...,0.165659,Somewhat-Bullish,MSFT,0.055046,0.101831,Neutral
22175,Experlogix Digital Commerce Expands into North...,20230831T163200,"SOUTH JORDAN, Utah, Aug. 31, 2023 /CNW/ - Expe...",0.470411,Bullish,MSFT,0.071357,0.088337,Neutral
22176,Experlogix Digital Commerce Expands into North...,20230831T163200,"SOUTH JORDAN, Utah, Aug. 31, 2023 /PRNewswire/...",0.470411,Bullish,MSFT,0.071357,0.088337,Neutral
22177,Amazon Rides High Amid Challenges In Cloud Com...,20230831T171300,Amazon Rides High Amid Challenges In Cloud Com...,0.207786,Somewhat-Bullish,MSFT,0.095327,0.333285,Somewhat-Bullish
22178,Baldur's Gate 3 Set to Arrive On Xbox By End O...,20230831T182710,Larian Studios has confirmed that the popular ...,0.203816,Somewhat-Bullish,MSFT,0.128822,-0.103902,Neutral


# Appending same day news data together

In [4]:
df = pd.read_csv(f"{ticker}_News_Content.csv")

# Filter rows with relevance_score > 0.5
df = df[df['relevance_score'] > 0.5]

# Extract date and time components
df['Date'] = pd.to_datetime(df['time_published'], format='%Y%m%dT%H%M%S').dt.date
df['Time'] = pd.to_datetime(df['time_published'], format='%Y%m%dT%H%M%S').dt.time

# Convert date to datetime format
df['Date'] = pd.to_datetime(df['Date'])

# Group by date and concatenate title and summary
df_news = df.groupby('Date').agg({'title': ' '.join, 'summary': ' '.join}).reset_index()

df_news.tail()

,Date,title,summary
441,2023-08-27,A Bull Market Is Coming: 2 Artificial Intellig...,Wall Street billionaire Ken Griffin is exceedi...
442,2023-08-28,"Not possible to slow tech pace, answer is in s...",AI - which will increasingly have a positive i...
443,2023-08-29,Microsoft Memo Leak Reveals Guidance For Manag...,Microsoft Corp. MSFT has made significant chan...
444,2023-08-30,Microsoft ( MSFT ) Gains But Lags Market: Wh...,"In the latest trading session, Microsoft (MSFT..."
445,2023-08-31,Microsoft to unbundle Teams software in Europe...,European Union regulators had in July opened a...


# Appending a price change column

In [5]:
df_price_change = pd.read_csv(f"{ticker}_Daily_Data.csv")

# Calculate the price change compared to the previous date
df_price_change['Price Change'] = df_price_change['Close'].diff()

# Add a new column to indicate if the price increased (1) or decreased (0)
df_price_change['Price Increase'] = (df_price_change['Price Change'] > 0).astype(int)

# Convert 'Date' column to datetime type in both dataframes
df_price_change['Date'] = pd.to_datetime(df_price_change['Date'])
df_news['Date'] = pd.to_datetime(df_news['Date'])

# Perform a left outer join based on the 'Date' column
merged_df = pd.merge(df_price_change, df_news, on='Date')

# Display the merged dataframe
merged_df.tail()

,Date,Open,High,Low,Close,Adj Close,Volume,Price Change,Price Increase,title,summary
356,2023-08-25,321.470001,325.359985,318.799988,322.980011,322.980011,21671400,3.010010,1,China-based 'Flax Typhoon' Hackers Targeting T...,China-based 'Flax Typhoon' Hackers Targeting T...
357,2023-08-28,325.660004,326.149994,321.721985,323.820007,323.820007,10588855,0.839996,1,"Not possible to slow tech pace, answer is in s...",AI - which will increasingly have a positive i...
358,2023-08-29,321.880005,328.983490,321.940002,327.500000,327.500000,10393681,3.679993,1,Microsoft Memo Leak Reveals Guidance For Manag...,Microsoft Corp. MSFT has made significant chan...
359,2023-08-30,328.670013,329.809998,326.450012,328.790009,328.790009,15205500,1.290009,1,Microsoft ( MSFT ) Gains But Lags Market: Wh...,"In the latest trading session, Microsoft (MSFT..."
360,2023-08-31,329.200012,330.909912,326.779999,327.950012,327.950012,11526643,-0.839996,0,Microsoft to unbundle Teams software in Europe...,European Union regulators had in July opened a...


In [6]:
# Shift the column by one row
shifted_column = merged_df['Price Increase'].shift(-1)

# Fill NaN values with 0
shifted_column = shifted_column.fillna(0)

# Convert to integers
shifted_column = shifted_column.astype(int)

# Add the shifted column back to the DataFrame
merged_df['shifted_Price_Increase'] = shifted_column

merged_df.head()

,Date,Open,High,Low,Close,Adj Close,Volume,Price Change,Price Increase,title,summary,shifted_Price_Increase
0,2022-03-07,288.529999,289.690002,278.529999,278.910004,275.015656,43157200,-10.949982,0,Microsoft announces setting up of its fourth d...,"Situated in Hyderabad, this will be the Micros...",0
1,2022-03-08,277.799988,283.959991,270.000000,275.850006,271.998413,48159500,-3.059998,0,Google to buy cybersecurity firm Mandiant for ...,Google's offer of $23 per share is at a premiu...,1
2,2022-03-09,283.440002,289.600006,280.779999,288.500000,284.471741,35204500,12.649994,1,Ralliton the First Company in Cyprus to Receiv...,"Ralliton, the award-winning managed cloud serv...",0
3,2022-03-10,283.019989,286.600006,280.579987,285.589996,281.602356,30628000,-2.910004,0,AvePoint Unveils Entrust to Help Organizations...,"JERSEY CITY, N.J., March 10, 2022 ( GLOBE N...",0
4,2022-03-11,287.959991,289.510010,279.429993,280.070007,276.159454,27209300,-5.519989,0,Artificial Intelligence in Supply Chain Market...,The market is driven by factors such as the ri...,0


# Sentiment analysis of collected news data

In [7]:
train_size = int(len(merged_df) * 0.8)
train, test = merged_df[0:train_size], merged_df[train_size:]

train.tail(10)

,Date,Open,High,Low,Close,Adj Close,Volume,Price Change,Price Increase,title,summary,shifted_Price_Increase
278,2023-04-26,296.700012,299.570007,292.730011,295.369995,294.103027,64599200,19.949982,1,"Microsoft reports 9% boost in profits, revenue...",Microsoft on Tuesday reported a 9% increase in...,1
279,2023-04-27,295.970001,305.200012,295.250000,304.829987,303.522430,46462600,9.459991,1,Ray Dalio Perplexed By Bitcoin's Popularity Vs...,Billionaire investor Ray Dalio has spoken out ...,1
280,2023-04-28,304.010010,308.929993,303.309998,307.260010,305.942047,36446700,2.430023,1,Microsoft is betting its future on cloud gamin...,U.K. regulators fears that Microsoft could in ...,0
281,2023-05-01,306.970001,308.600006,305.149994,305.559998,304.249329,21294100,-1.700012,0,Microsoft Corporation ( MSFT ) is Attracting...,Microsoft (MSFT) has received quite a bit of a...,0
282,2023-05-02,307.760010,309.179993,303.910004,305.410004,304.099945,26404400,-0.149994,0,Apple Vs Microsoft: Safari Overtakes Edge As M...,Apple Inc.'s AAPL Safari overtook Microsoft Co...,0
283,2023-05-03,306.619995,308.609985,304.089996,304.399994,303.094299,22360800,-1.010010,0,"CEOs of Microsoft, Alphabet called to AI meeti...",Vice President Kamala Harris will reportedly h...,1
284,2023-05-04,306.239990,307.760010,303.399994,305.410004,304.099945,22519900,1.010010,1,"AI-powered Bing ditches waitlist, adds images ...",A screenshot of the AI-powered Bing delivering...,1
285,2023-05-05,305.720001,311.970001,304.269989,310.649994,309.317505,28181200,5.239990,1,Check Out What Whales Are Doing With MSFT - Mi...,A whale with a lot of money to spend has taken...,0
286,2023-05-08,310.130005,310.200012,306.089996,308.649994,307.326050,21318600,-2.000000,0,Quisitive Sets First Quarter 2023 Earnings Cal...,"TORONTO, May 08, 2023 ( GLOBE NEWSWIRE ) -- Qu...",0
287,2023-05-09,308.000000,310.040009,306.309998,307.000000,305.683136,21340800,-1.649994,0,Google Cloud makes AI 'land grab' with wave of...,Alphabet Inc.'s Google Cloud on Tuesday made i...,1


In [8]:
#preparing the train dataset

#Removing punctuations of title and summary columns
df_train = train.iloc[:, :11]
df_train = df_train.replace("[^a-zA-Z]", " ", regex=True)

# Convert 'title' column to lowercase
df_train['title'] = df_train['title'].str.lower()
# Convert 'summary' column to lowercase
df_train['summary'] = df_train['summary'].str.lower()

train_summary_list = df_train['summary'].tolist()

print(train_summary_list[0])

situated in hyderabad  this will be the microsoft s largest data centre in the country shares of mandiant  a      billion cybersecurity firm  closed up     after a report said google is interested in acquiring the company  google    nasdaq  googl    is in talks to purchase cybersecurity behemoth mandiant    nasdaq  mndt     the information reported monday  the cybersecurity stock took off late in the regular trading session after the report was published  and mandiant ended the trading day        higher at          th


In [9]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.ensemble import RandomForestClassifier

#implementing bag-of-words
countVector = CountVectorizer(ngram_range=(3,3))
trainDataset = countVector.fit_transform(train_summary_list)

In [10]:
#implementing random forest classifier
randomClassifier = RandomForestClassifier(n_estimators=200, criterion='entropy')
randomClassifier.fit(trainDataset, train['shifted_Price_Increase'])

RandomForestClassifier(criterion='entropy', n_estimators=200)

In [11]:
#predict for the test dataset

#Removing punctuations of title and summary columns
df_test = test.iloc[:, :11]
df_test = df_test.replace("[^a-zA-Z]", " ", regex=True)

# Convert 'title' column to lowercase
df_test['title'] = df_test['title'].str.lower()
# Convert 'summary' column to lowercase
df_test['summary'] = df_test['summary'].str.lower()

test_summary_list = df_test['summary'].tolist()

print(test_summary_list[0])

twitter gets new dm feature  voice and video chat to be added soon     business standard     the deal signifies a further bid by microsoft to ramp up its efforts in ai  which has become a key focus for the company  indian company helps clients make apps without any coding experience     in a deal believed to be the first commercial agreement for fusion power  the tech giant has agreed to purchase electricity from helion energy a startup backed by openai founder sam altman within about five years  microsoft corp msft shares are trading higher wednesday after it agreed to purchase electricity from startup helion energy in       marking the first commercial deal for fusion power  constellation energy corp ceg will be the power marketer and manage project transmission  microsoft is quickly becoming the best stock to invest in for exposure to artificial intelligence  now that it s in cost saving mode  microsoft is foregoing salary increases this year after giving raises to some employees in

In [12]:
testDataset = countVector.transform(test_summary_list)
predictions = randomClassifier.predict(testDataset)
print(predictions)

[0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]


In [13]:
#import library to check the accuracy
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score

matrix = confusion_matrix(test['shifted_Price_Increase'], predictions)
print(matrix)
score = accuracy_score(test['shifted_Price_Increase'], predictions)
print(score)
report = classification_report(test['shifted_Price_Increase'], predictions)
print(report)

[[32  0]
 [40  1]]
0.4520547945205479
              precision    recall  f1-score   support

           0       0.44      1.00      0.62        32
           1       1.00      0.02      0.05        41

    accuracy                           0.45        73
   macro avg       0.72      0.51      0.33        73
weighted avg       0.76      0.45      0.30        73



# Creating a new dataframe with prices and news data concatenated

In [119]:
# import pandas as pd

# # Assuming df1 and df2 are your two dataframes
# # You can replace these with your actual dataframes
# df1 = df_price
# df2 = df_news

# # Convert 'Date' column to datetime type in both dataframes
# df_price['Date'] = pd.to_datetime(df_price['Date'])
# df_news['Date'] = pd.to_datetime(df_news['Date'])

# # Perform a left outer join based on the 'Date' column
# merged_df = pd.merge(df_price, df_news, on='Date', how='left')

# # Display the merged dataframe
# merged_df.tail()

# Azure connection

In [8]:
# !pip install azure-functions
# !pip install azure-storage-blob

In [13]:
import yfinance as yf
import pandas as pd
import datetime
import os
import schedule
import time
from azure.storage.blob import BlobServiceClient, BlobClient, ContainerClient

# Azure Blob Storage credentials
azure_storage_connection_string = "DefaultEndpointsProtocol=https;AccountName=sheetcsvblob12;AccountKey=nMn+RPU7dlQJAgA51+V8YQ3I1XMPD/4z8uCZb/YFF3o45zKk7yXatxegaCVElUsuJVHASuaeOB/8+AStqDwNTA==;EndpointSuffix=core.windows.net"
container_name = "sheet-01-test"

def fetch_daily_data(ticker, start_date, end_date):
    df = yf.download(ticker, start=start_date, end=end_date, interval='1d')
    return df

def update_daily_data_for_ticker(ticker):
    # Initialize the Azure Blob Service Client
    blob_service_client = BlobServiceClient.from_connection_string(azure_storage_connection_string)
    container_client = blob_service_client.get_container_client(container_name)
    
    # Determine the CSV file name
    csv_file_name = ticker + '_Daily_Data.csv'

    if not container_client.get_blob_client(csv_file_name).exists():
        # Fetch data from '2013-01-01' as initial data
        end_date = datetime.datetime.now().date()
        start_date = '2023-05-01'
        daily_data = fetch_daily_data(ticker, start_date, end_date)
        
        # Reset index and extract date as a separate column
        daily_data.reset_index(inplace=True)
        daily_data['Date'] = daily_data['Date'].dt.date
        
        # Save the DataFrame to a CSV file if the file doesn't exist
        csv_content = daily_data.to_csv(index=False)
        container_client.get_blob_client(csv_file_name).upload_blob(csv_content)
        print(f"{len(daily_data)} new records added to a new {csv_file_name} file.")
    else:
        # Load existing data from the CSV stored in Azure Blob Storage
        blob_client = container_client.get_blob_client(csv_file_name)
        blob_content = blob_client.download_blob()
        existing_df = pd.read_csv(blob_content.content_as_text())
        
        # Get the last date from the existing data
        latest_date = pd.to_datetime(existing_df['Date']).max()
        
        # Convert the latest_date to a Pandas Timestamp
        latest_date = pd.Timestamp(latest_date)
        
        # Define date range for fetching new data
        end_date = datetime.datetime.now().date()
        start_date = (latest_date + datetime.timedelta(days=1)).date()
        
        if start_date < end_date:
            # Fetch daily data
            daily_data = fetch_daily_data(ticker, start_date, end_date)
            
            # Reset index and extract date as a separate column
            daily_data.reset_index(inplace=True)
            daily_data['Date'] = daily_data['Date'].dt.date
            
            # Append new data to the CSV file in Azure Blob Storage
            updated_df = pd.concat([existing_df, daily_data], ignore_index=True)
            csv_content = updated_df.to_csv(index=False)
            blob_client.upload_blob(csv_content, overwrite=True)
            
            print(f"{len(daily_data)} new records added to the {csv_file_name} file.")
        else:
            print(f"No new records to add for {ticker}.")

# List of stock symbols
symbols = ["AAPL", "ORCL", "MSFT"]  # Add more symbols as needed

# Function to perform the initial data update for all symbols
def initial_data_update():
    for symbol in symbols:
        update_daily_data_for_ticker(symbol)

# Schedule the automatic update daily
for symbol in symbols:
    schedule.every().day.at("22:48").do(update_daily_data_for_ticker, symbol)

# Perform the initial data update
initial_data_update()

# Run the scheduled tasks
while True:
    schedule.run_pending()
    time.sleep(1)  # Sleep for 24 hours

[*********************100%***********************]  1 of 1 completed
106 new records added to a new AAPL_Daily_Data.csv file.
[*********************100%***********************]  1 of 1 completed
106 new records added to a new ORCL_Daily_Data.csv file.
[*********************100%***********************]  1 of 1 completed
106 new records added to a new MSFT_Daily_Data.csv file.


FileNotFoundError: [Errno 2] No such file or directory: 'Date,Open,High,Low,Close,Adj Close,Volume\r\n2023-05-01,169.27999877929688,170.4499969482422,168.63999938964844,169.58999633789062,169.1273651123047,52472900\r\n2023-05-02,170.08999633789062,170.35000610351562,167.5399932861328,168.5399932861328,168.08021545410156,48425700\r\n2023-05-03,169.5,170.9199981689453,167.16000366210938,167.4499969482422,166.99319458007812,65136000\r\n2023-05-04,164.88999938964844,167.0399932861328,164.30999755859375,165.7899932861328,165.3377227783203,81235400\r\n2023-05-05,170.97999572753906,174.3000030517578,170.75999450683594,173.57000732421875,173.0965118408203,113316400\r\n2023-05-08,172.47999572753906,173.85000610351562,172.11000061035156,173.5,173.0266876220703,55962800\r\n2023-05-09,173.0500030517578,173.5399932861328,171.60000610351562,171.77000427246094,171.30142211914062,45326900\r\n2023-05-10,173.02000427246094,174.02999877929688,171.89999389648438,173.55999755859375,173.08653259277344,53724

In [ ]:
//google sheet----data (google cloud ekak hadanna) use cloud sheet API